In [95]:
import math
import json
import requests
import os
import pandas as pd
import mdutils
import urllib.parse
from wikidataintegrator import wdi_core
import graphviz

In [3]:
project = "wikiproject-biodiversity"
license = "cc0,cc-by"
language = "en"

In [4]:
url = f"https://api.inaturalist.org/v1/observations/species_counts?photo_license={license}&quality_grade=research&per_page=500&project_id={project}"
print(url)

https://api.inaturalist.org/v1/observations/species_counts?photo_license=cc0,cc-by&quality_grade=research&per_page=500&project_id=wikiproject-biodiversity


In [5]:
result = json.loads(requests.get(url).text)
for page in range(1, math.ceil(result["total_results"]/500)+1):
    try:
        print(page)
        nextpageresult = json.loads(requests.get(url+"&page="+str(page)).text)
        if "results" in nextpageresult:
            for obs in nextpageresult["results"]:
                result["results"].append(obs)
    except Exception as e:
        print(e)
        print(url+"&page="+str(page))
        continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [6]:
table = dict()
for record in result["results"]:
    if record["taxon"]["id"] not in table.keys():
        table[record["taxon"]["id"]] = dict()
    table[record["taxon"]["id"]]["taxon_name"] = record["taxon"]["name"]

to_verify = []
for taxon in table.keys():
    to_verify.append(table[taxon]['taxon_name'])


In [12]:
from urllib.parse import quote
def verify_wikidata(taxon_names):
    verified = []
    i = 1
    for chunks in [taxon_names[i:i + 50] for i in range(0, len(taxon_names), 50)]:
        names=" ".join('"{0}"'.format(w.replace("\"", "")) for w in chunks)
        query = f"""
             SELECT DISTINCT ?taxon_name (COUNT(?item) AS ?item_count) (COUNT(DISTINCT ?article) AS ?article_count) (GROUP_CONCAT(DISTINCT ?article) as ?wparticles) (GROUP_CONCAT(DISTINCT ?schema) as ?patterns ) WHERE {{
                VALUES ?taxon_name {{{names}}}

                {{
                ## A wikipedia article that is linked through a different item that is labeled as
                ## instance of (P31) organisms known by a particular common name (Q55983715). The link
                ## with the given taxon name is through qualifier of (P642).
                BIND ("organisms known by a particular common name (e:E382)" as ?schema)
                ?qitem wdt:P225 ?taxon_name .
                ?item p:P31 [
                     ps:P31 wd:Q55983715 ;
                     pq:P642 ?qitem ;
                  ] .
                OPTIONAL {{?article schema:about ?item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                ## A wikipedia article that is directly linked through its taxon name
                BIND ("direct link (e:E383)" as ?schema)
                ?item wdt:P225 ?taxon_name .
                OPTIONAL {{?article schema:about ?item ;
                            schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                    ## A wikipedia article that is linked through a basionym
                    BIND ("basionym (e:E384)" as ?schema)
                    ?item wdt:P566 ?basionym ;
                          wdt:P225 ?taxon_name .
                    OPTIONAL {{?article schema:about ?basionym ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                    ## A wikipedia article that is linked through a taxonomic synonym
                    BIND ("taxon synonym 1 (e:E385)" as ?schema)
                    ?item wdt:P1420 ?synonym ;
                          wdt:P225 ?taxon_name .
                    OPTIONAL {{?article schema:about ?synonym ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                  ## A Wikipedia article that is linked through a synonym, this is different from
                  ## the above one where the taxon synonym is through a property taxon synonym
                  ## here it is through a P31 (instance of) synonym.
                    BIND ("taxon synonym 2 (e:E386)" as ?schema)
                   ?qitem wdt:P225 ?taxon_name .
                   ?item p:P31 [
                      ps:P31 wd:Q1040689 ;
                      pq:P642 ?qitem
                   ]
                   OPTIONAL {{?article schema:about ?item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                    ## a wikipedia article that is linked through its protonym
                    BIND ("protonym (e:E387)" as ?schema)
                  ?qitem wdt:P225 ?taxon_name .
                  ?item p:P2868 [
                               ps:P2868 wd:Q10640897 ; # has role protonym
                               pq:P642 ?qitem ;
                         ] .

                OPTIONAL {{?article schema:about ?item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                  ## a wikipedia article that is linked through a property said to be the same as (P460)
                  BIND ("said to be the same (e:E388)" as ?schema)
                  ?item wdt:P225 ?taxon_name .
                  ?wikidata_item wdt:P460 ?item .
                  OPTIONAL {{?article schema:about ?wikidata_item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}}}
            }} GROUP BY ?taxon_name
            """
        # print(query)
        try:
            results = wdi_core.WDFunctionsEngine.execute_sparql_query(query)

            for result in results["results"]["bindings"]:
                #print(result["taxon_name"], result["article_count"]["value"])
                if int(result["article_count"]["value"])>1:
                    print(result["taxon_name"]["value"], ":", result["article_count"]["value"], result["wparticles"]["value"], result["patterns"]["value"])
                # print(result["article_count"]["value"])
                if result["article_count"]["value"]=='0':
                    verified.append(result["taxon_name"]["value"])
        except Exception as e:
            print(url)
            print(e)
            continue
    return verified

In [8]:
len(to_verify)

8126

In [9]:
verified = verify_wikidata(to_verify)
verified

Hirundo rustica : 2 https://en.wikipedia.org/wiki/Barn_swallow https://en.wikipedia.org/wiki/Hirundo direct link (e:E383) basionym (e:E384) said to be the same (e:E388)
Anemonoides nemorosa : 2 https://en.wikipedia.org/wiki/Anemonoides_nemorosa https://en.wikipedia.org/wiki/Anemone_nemorosa direct link (e:E383) basionym (e:E384) taxon synonym 2 (e:E386)
Felis catus : 2 https://en.wikipedia.org/wiki/Cat https://en.wikipedia.org/wiki/Felis_catus organisms known by a particular common name (e:E382) direct link (e:E383) taxon synonym 1 (e:E385)
Canis familiaris : 2 https://en.wikipedia.org/wiki/Dog https://en.wikipedia.org/wiki/Dingo organisms known by a particular common name (e:E382) direct link (e:E383)
Stellaria media : 2 https://en.wikipedia.org/wiki/Stellaria_media https://en.wikipedia.org/wiki/Stellaria_apetala direct link (e:E383) basionym (e:E384) taxon synonym 1 (e:E385)
Polyommatus icarus : 2 https://en.wikipedia.org/wiki/Common_blue https://en.wikipedia.org/wiki/Polyommatus_and

['Corvus monedula',
 'Pseudofumaria lutea',
 'Rhinella alata',
 'Veronica urticifolia',
 'Otospermophilus beecheyi',
 'Pentaglottis sempervirens',
 'Prenanthes purpurea',
 'Polytrichum formosum',
 'Eucnide lobata',
 'Bryonia cretica',
 'Pentanema squarrosum',
 'Antigone canadensis',
 'Ortilia ithra',
 'Anthomyia illocata',
 'Hyla squirella',
 'Knautia dipsacifolia',
 'Baccharis trimera',
 'Oblada melanurus',
 'Solidago chilensis',
 'Rhopalapion longirostre',
 'Petasites paradoxus',
 'Mnium hornum',
 'Hydrobates melania',
 'Orobanche gracilis',
 'Euphaea ochracea',
 'Valeriana tripteris',
 'Diplacus aurantiacus',
 'Polyura bharata',
 'Jacobaea alpina',
 'Austroeupatorium inulifolium',
 'Lactophrys triqueter',
 'Eriothrix rufomaculata',
 'Coccothraustes vespertinus',
 'Pachliopta aristolochiae',
 'Sibovia sagata',
 'Camponotus rufipes',
 'Hydrobates homochroa',
 'Palicourea tomentosa',
 'Tabanus atratus',
 'Ilexia intractata',
 'Condylostylus longicornis',
 'Dascillus cervinus',
 'Diclip

In [10]:
len(verified)

1011

In [11]:
import time
photo_collection = []
i = 0
for taxon_name in verified:
    time.sleep(1)
    try:
        photos_url = f"https://api.inaturalist.org/v1/observations?photo_license={license}&project_id={project}&taxon_name={taxon_name}&order=desc&order_by=created_at"
        results = json.loads(requests.get(photos_url).text)
        i+=1
        print(str(i),taxon_name)
        for record in results["results"]:
            if record["taxon"]["id"] not in table.keys():
                continue
            photo_collection.append(record)
            #for photo in record["observation_photos"]:
            #    if "photos" not in table[record["taxon"]["id"]].keys():
            #        table[record["taxon"]["id"]]["photos"] = {}
                
            #    table[record["taxon"]["id"]]["photos"].append(photo["photo"]["url"])
    except Exception as e:
        print(e)
        print(url+"&page="+str(page))
        


1 Corvus monedula
2 Pseudofumaria lutea
3 Rhinella alata
4 Veronica urticifolia
5 Otospermophilus beecheyi
6 Pentaglottis sempervirens
7 Prenanthes purpurea
8 Polytrichum formosum
9 Eucnide lobata
10 Bryonia cretica
11 Pentanema squarrosum
12 Antigone canadensis
13 Ortilia ithra
14 Anthomyia illocata
15 Hyla squirella
16 Knautia dipsacifolia
17 Baccharis trimera
18 Oblada melanurus
19 Solidago chilensis
20 Rhopalapion longirostre
21 Petasites paradoxus
22 Mnium hornum
23 Hydrobates melania
24 Orobanche gracilis
25 Euphaea ochracea
26 Valeriana tripteris
27 Diplacus aurantiacus
28 Polyura bharata
29 Jacobaea alpina
30 Austroeupatorium inulifolium
31 Lactophrys triqueter
32 Eriothrix rufomaculata
33 Coccothraustes vespertinus
34 Pachliopta aristolochiae
35 Sibovia sagata
36 Camponotus rufipes
37 Hydrobates homochroa
38 Palicourea tomentosa
39 Tabanus atratus
40 Ilexia intractata
41 Condylostylus longicornis
42 Dascillus cervinus
43 Dicliptera squarrosa
44 Apoica gelida
45 Phyteuma nigrum

348 Micrathyria atra
349 Arundina graminifolia
350 Silvetia compressa
351 Cladophora columbiana
352 Antrodia heteromorpha
353 Parmotrema reticulatum
354 Orphulella concinnula
355 Nereocystis luetkeana
356 Eumaeus childrenae
357 Pelvetiopsis limitata
358 Comibaena pictipennis
359 Inocybe lanuginosa
360 Cicindela haemorrhagica
361 Calyptridium umbellatum
362 Achlyodes pallida
363 Valeriana montana
364 Eriphia smithii
365 Hyptis mutabilis
366 Globularia nudicaulis
367 Anomala dubia
368 Agelas sventres
369 Aiolochroia crassa
370 Sida ulmifolia
371 Lutjanus mahogoni
372 Callyspongia fallax
373 Nalassus laevioctostriatus
374 Gomphrena serrata
375 Aplysina lacunosa
376 Castilleja arvensis
377 Philonotis fontana
378 Cacoecimorpha pronubana
379 Crotalaria pumila
380 Piper peltatum
381 Polypodium interjectum
382 Desmia maculalis
383 Glossonotus acuminatus
384 Orochelidon murina
385 Eurybia patrona
386 Dasymutilla klugii
387 Xenodon dorbignyi
388 Tridax coronopifolia
389 Magneuptychia libye
390 U

693 Picnomon acarna
694 Carthamus tenuis
695 Teucrium capitatum
696 Ctenophora flaveolata
697 Acromis sparsa
698 Salticus mutabilis
699 Coraliomela quadrimaculata
700 Staphylinus dimidiaticornis
701 Xerula pudens
702 Leukoma crassicosta
703 Venerupis largillierti
704 Segestria bavarica
705 Oedemera femorata
706 Platyrhinus resinosus
707 Deltanus texanus
708 Ctenicera pectinicornis
709 Esperanza texana
710 Selatosomus gravidus
711 Silis ruficollis
712 Agriotes pilosellus
713 Liparus germanus
714 Limonium pruinosum
715 Phragmidium rosae-multiflorae
716 Molorchus minor
717 Sapygina decemguttata
718 Dufourea ligulata
719 Cryptocephalus sexpunctatus
720 Ropalidia cyathiformis
721 Anaspis maculata
722 Pedicularis kerneri
723 Xysticus kochi
724 Macrosaccus robiniella
725 Xanthogramma dives
726 Dodecastichus geniculatus
727 Alopecosa barbipes
728 Orsilochides stictica
729 Bidens subalternans
730 Belonocnema treatae
731 Sericomyia bombiformis
732 Gentiana terglouensis
733 Polygonum maritimum
73

In [71]:
# User
# print(photo_collection[1]["user"])
# Taxon
print(photo_collection[1]["taxon"]["id"])
#print(photo_collection[1].keys())
#print(photo_collection[1]["observation_photos"])

8000


In [76]:
collection = {}
for observation in photo_collection:
    if observation["taxon"]["name"] in verified:
        if observation["taxon"]["id"] not in collection.keys():
            collection[observation["taxon"]["id"]] = dict()
            collection[observation["taxon"]["id"]]["name"] = observation["taxon"]["name"]
            collection[observation["taxon"]["id"]]["observers"] = dict()
        if observation["user"]["id"] not in  collection[observation["taxon"]["id"]]["observers"].keys():
                collection[observation["taxon"]["id"]]["observers"][observation["user"]["id"]] = dict()
        if observation["user"]['name'] != '':
                collection[observation["taxon"]["id"]]["observers"][observation["user"]["id"]]["name"] =  observation["user"]['name'] 
        if observation["user"]['login'] != '':
                collection[observation["taxon"]["id"]]["observers"][observation["user"]["id"]]["login"] =  observation["user"]['login'] 
        
        collection[observation["taxon"]["id"]]["observers"][observation["user"]["id"]]["observations"] = observation["observation_photos"]      
collection

{8000: {'name': 'Corvus monedula',
  'observers': {488776: {'name': 'Andra Waagmeester',
    'login': 'andrawaag',
    'observations': [{'id': 175691463,
      'position': 0,
      'uuid': '538b5354-9ea4-4b76-b170-92480b4325d3',
      'photo': {'id': 188288838,
       'license_code': 'cc0',
       'original_dimensions': {'width': 1740, 'height': 1305},
       'url': 'https://inaturalist-open-data.s3.amazonaws.com/photos/188288838/square.jpeg',
       'attribution': 'no rights reserved',
       'flags': []}}]},
   1002688: {'login': 'kitbeard',
    'observations': [{'id': 78229423,
      'position': 0,
      'uuid': '1a30e34c-a4e2-4b2b-a478-165895e71237',
      'photo': {'id': 84035491,
       'license_code': 'cc-by',
       'original_dimensions': {'width': 710, 'height': 730},
       'url': 'https://inaturalist-open-data.s3.amazonaws.com/photos/84035491/square.jpeg',
       'attribution': '(c) kitbeard, some rights reserved (CC BY)',
       'flags': []}}]},
   419580: {'name': 'Paul Br

In [139]:
def extract_wikidata_taxonomic_schema(name, language="en"):
    query = f"""
     SELECT DISTINCT ?schema ?subject ?subjectLabel ?object ?objectLabel (GROUP_CONCAT(DISTINCT ?object_wikifamily) as ?objectwikis) (GROUP_CONCAT(DISTINCT ?subject_wikifamily) as ?subjectwikis) WHERE {{
       SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
       VALUES ?taxon_name {{"{name}"}}
       # VALUES ?object_wikifamily {{<https://en.wikipedia.org/>}}
       # VALUES ?subject_wikifamily {{<https://en.wikipedia.org/>}}
        {{
        ## A wikipedia article that is linked through a different item that is labeled as
        ## instance of (P31) organisms known by a particular common name (Q55983715). The link
        ## with the given taxon name is through qualifier of (P642).
        BIND ("organisms known by a particular common name (e:E382)" as ?schema)
        BIND (?aka_item as ?subject)
        BIND (?item as ?object)

        optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}

        ?item wdt:P225 ?taxon_name .
        ?aka_item p:P31 [
             ps:P31 wd:Q55983715 ;
             pq:P642 ?item ;
          ] .
        optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
        }}

        UNION

        {{
            ## A wikipedia article that is linked through a basionym
            BIND ("basionym (e:E384)" as ?schema)
            BIND (?item as ?subject)
            BIND (?basionym as ?object)
            ?item wdt:P566 ?basionym ;
                  wdt:P225 ?taxon_name .
                optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
        }}

        UNION

        {{
            ## A wikipedia article that is linked through a taxonomic synonym
            BIND ("taxon synonym 1 (e:E385)" as ?schema)
            BIND (?item as ?subject)
            BIND (?synonym as ?object)
            ?item wdt:P1420 ?synonym ;
                  wdt:P225 ?taxon_name .
                optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
        }}

        UNION

        {{
          ## A Wikipedia article that is linked through a synonym, this is different from
          ## the above one where the taxon synonym is through a property taxon synonym
          ## here it is through a P31 (instance of) synonym.
            BIND ("taxon synonym 2 (e:E386)" as ?schema)
            BIND (?synonym_item as ?subject)
            BIND (?item as ?object)
           ?item wdt:P225 ?taxon_name .
           ?synonym_item p:P31 [
              ps:P31 wd:Q1040689 ;
              pq:P642 ?item
           ]
               optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
        }}

        UNION

        {{
            ## a wikipedia article that is linked through its protonym
            BIND ("protonym (e:E387)" as ?schema)
            BIND (?protonym_item as ?subject)
            BIND (?item as ?object)
          ?item wdt:P225 ?taxon_name .
          ?protonym_item p:P2868 [
                       ps:P2868 wd:Q10640897 ; # has role protonym
                       pq:P642 ?item ;
                 ] .
                optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
        }}

        UNION

        {{
          ## a wikipedia article that is linked through a property said to be the same as (P460)
          BIND ("said to be the same (e:E388)" as ?schema)
          BIND (?wikidata_item as ?subject)
          BIND (?item as ?object)
          ?item wdt:P225 ?taxon_name .
          ?wikidata_item wdt:P460 ?item .
              optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
    }}
    }} GROUP BY ?schema ?subject ?subjectLabel ?object ?objectLabel
    """
    edgesdrawn = []
    df = wdi_core.WDFunctionsEngine.execute_sparql_query(query, as_dataframe=True)
    if len(df) >0:
        schema = graphviz.Digraph(name, filename=f"./wikidata_schemas/{name.replace(' ', '_')}.gv")
        graphviz.Digraph()
        schema.graph_attr['label'] = name
        schema.graph_attr['rankdir'] = 'LR'
        schema.attr('node', shape="rectangle")
        nodes = []
        for index, row in df.iterrows():
            subjectnode = str(row["subject"]).replace("http://www.wikidata.org/entity/", "")
            if subjectnode not in nodes:
                nodes.append(subjectnode)
                schema.node(subjectnode, label=row["subjectLabel"]+' (wd:'+subjectnode+")")

            objectnode = str(row["object"]).replace("http://www.wikidata.org/entity/", "")
            if objectnode not in nodes:
                nodes.append(objectnode)
                schema.node(objectnode, label=row["objectLabel"]+' (wd:'+row["object"].replace("http://www.wikidata.org/entity/", "")+")")

            if f"https://{language}.wikipedia.org/" in row["subjectwikis"].split():
                if subjectnode+nodename not in edgesdrawn:
                    nodename = f"{language}.wiki"
                    schema.node(nodename)
                    schema.edge(subjectnode, nodename, label="schema:isPartOf")
                    edgesdrawn.append(subjectnode+nodename)
            if f"https://{language}.wikipedia.org/" in row["objectwikis"].split():
                 if objectnode+nodename not in edgesdrawn:
                    schema.node(nodename)
                    schema.edge(objectnode, nodename, label="schema:isPartOf")    
                    edgesdrawn.append(objectnode+nodename)
            schema.edge(subjectnode, objectnode, label=row["schema"])
        schema.render(f"./wikidata_schemas/{name.replace(' ', '_')}.gv", format="png")
    

In [154]:
for taxon in collection.keys():
    extract_wikidata_taxonomic_schema(collection[taxon]["name"])

    

['Megatoma_undata.gv.png',
 'Palicourea_tomentosa.gv',
 'Calandrinia_menziesii.gv',
 'Justicia_comata.gv',
 'Modiola_caroliniana.gv.png',
 'Murdannia_nudiflora.gv.png',
 'Ricciocarpos_natans.gv.png',
 'Lactophrys_triqueter.gv',
 'Euryops_abrotanifolius.gv.png',
 'Calogaya_decipiens.gv.png',
 'Pentaglottis_sempervirens.gv',
 'Solanum_amygdalifolium.gv',
 'Phyla_strigulosa.gv.png',
 'Aristocypha_trifasciata.gv.png',
 'Ludwigia_leptocarpa.gv',
 'Puccinia_urticata.gv',
 'Trichocereus_candicans.gv',
 'Antrodia_heteromorpha.gv',
 'Hypochaeris_albiflora.gv',
 'Polyommatus_vanensis.gv',
 'Jacobaea_alpina.gv',
 'Torenia_crustacea.gv.png',
 'Physcia_tenella.gv.png',
 'Helichrysum_grandiflorum.gv.png',
 'Hydrobates_melania.gv.png',
 'Gulosus_aristotelis.gv.png',
 'Parapolystichum_glabellum.gv',
 'Hemipholiota_populnea.gv',
 'Pachliopta_aristolochiae.gv',
 'Aristocypha_quadrimaculata.gv',
 'Colletia_paradoxa.gv.png',
 'Conocephalus_albescens.gv',
 'Calyptridium_umbellatum.gv.png',
 'Priva_lappulac

In [151]:
import shutil
shutil.rmtree('./new_articles/')
#shutil.rmtree('./wikidata_schemas/')
os.makedirs(os.path.dirname('./new_articles/'), exist_ok=True)
#os.makedirs(os.path.dirname('./wikidata_schemas/'), exist_ok=True)

In [155]:
for taxon in collection.keys():
    mdFile = mdutils.MdUtils(file_name='new_articles/'+collection[taxon]["name"].replace(" ", "_"),title=collection[taxon]["name"])
    mdFile.new_line(f"[iNaturalist taxon id: {taxon}](https://www.inaturalist.org/taxa/{taxon})")
    schema_file_name = collection[taxon]["name"].replace(" ", "_")+".gv.png"

    if schema_file_name in os.listdir('./wikidata_schemas/'):
        print(schema_file_name,"im here")
        mdFile.new_header(level=1, title="Taxonomy in Wikidata")
        mdFile.new_line(mdFile.new_inline_image(text=collection[taxon]["name"], path="../wikidata_schemas/"+schema_file_name))
    mdFile.new_header(level=1, title="Photos")
    
    for observer in collection[taxon]["observers"].keys():
        if 'name' in collection[taxon]["observers"][observer].keys():
            mdFile.new_header(level=2, title="by: "+collection[taxon]["observers"][observer]["name"])
        else:
            mdFile.new_header(level=2, title="by: "+collection[taxon]["observers"][observer]["login"])
        for photo in collection[taxon]["observers"][observer]["observations"]:
            mdFile.new_line(f"![taxon{photo['id']}]({photo['photo']['url'].replace('square', 'medium')})")
    mdFile.create_md_file()

Corvus_monedula.gv.png im here
Otospermophilus_beecheyi.gv.png im here
Pentaglottis_sempervirens.gv.png im here
Pentanema_squarrosum.gv.png im here
Antigone_canadensis.gv.png im here
Rhopalapion_longirostre.gv.png im here
Petasites_paradoxus.gv.png im here
Mnium_hornum.gv.png im here
Hydrobates_melania.gv.png im here
Jacobaea_alpina.gv.png im here
Austroeupatorium_inulifolium.gv.png im here
Lactophrys_triqueter.gv.png im here
Coccothraustes_vespertinus.gv.png im here
Pachliopta_aristolochiae.gv.png im here
Hydrobates_homochroa.gv.png im here
Palicourea_tomentosa.gv.png im here
Condylostylus_longicornis.gv.png im here
Pentanema_verbascifolium.gv.png im here
Gentianopsis_ciliata.gv.png im here
Plagiomnium_undulatum.gv.png im here
Chromacris_colorata.gv.png im here
Herpothallon_rubrocinctum.gv.png im here
Drimia_numidica.gv.png im here
Erythrostemon_gilliesii.gv.png im here
Priva_lappulacea.gv.png im here
Dacrymyces_stillatus.gv.png im here
Pygochelidon_cyanoleuca.gv.png im here
Ampelaste

In [150]:
import yaml

with open(r'_toc.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    species = yaml.load(file, Loader=yaml.FullLoader)

species["parts"][0]["chapters"] = []
for file in os.listdir('./new_articles/'):
    species["parts"][0]["chapters"].append({'file': 'new_articles/'+file})
with open(r'_toc.yml', 'w+') as file:
    documents = yaml.dump(species, file)

In [139]:
from ghp_import import ghp_import
ghp_import("_build/html", nojekyll=True, push=True, force=True)

In [66]:
!ls new_articles

Abia_fasciata.md                       Leptotarsus_zeylandiae.md
Abrochia_fulvisphex.md                 Lessonia_variegata.md
Abutilon_hypoleucum.md                 Lestranicus_transpectus.md
Acalypha_monostachya.md                Leucanthemopsis_alpina.md
Acanalonia_servillei.md                Leucauge_fastigata.md
Acanonicus_laticornis.md               Leucauge_volupis.md
Acanthostracion_polygonium.md          Leucochimona_vestalis.md
Acanthostyles_buniifolius.md           Leucochrysa_explorata.md
Aceria_cephalonea.md                   Leukoma_crassicosta.md
Aceria_echii.md                        Limacellopsis_guttata.md
Aceria_exigua.md                       Limnephilus_auricula.md
Aceria_macrochela.md                   Limonium_cancellatum.md
Aceria_parulmi.md                      Limonium_pruinosum.md
Achlyodes_pallida.md                   Limonium_puberulum.md
Aconitum_vulparia.md                   Lindernia_rotundifolia.md
Acontia_titania.md                     Linum_viscosum.md

In [68]:
!cat new_articles/Abia_fasciata.md


Abia fasciata

# Abia fasciata
  
[iNaturalist taxon id: Abia fasciata](https://www.inaturalist.org/taxa/Abia fasciata)
# Photos

## by: Tobias Schönberg
  
![taxon203800689](https://inaturalist-open-data.s3.amazonaws.com/photos/218315494/medium.jpg)  
![taxon203800690](https://inaturalist-open-data.s3.amazonaws.com/photos/218315460/medium.jpg)